In [1]:
from scipy.signal import butter, filtfilt
import numpy as np
from pydub import AudioSegment
import pandas as pd
import tqdm
import os

def low_pass_filter(data, cutoff_freq, sample_rate, order=10):
    nyquist_rate = sample_rate / 2.0
    normal_cutoff = cutoff_freq / nyquist_rate
    
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data


In [2]:
csv_file = "/home/pdnguyen/Audio_Augmentation/fubon_11_10_2024/metadata.csv"
out_dir = "/home/pdnguyen/Audio_Augmentation/fubon_bandpass_filter"
df_fubong = pd.read_csv(csv_file)
with open(f'{out_dir}/metadata.csv', 'w', encoding='utf8') as fp:
    print(f'file,text,duration', file=fp)
    for index, row in tqdm.tqdm(df_fubong.iterrows()):
        if not os.path.exists(f"{out_dir}/wavs"):
            os.makedirs(f"{out_dir}/wavs")
        new_path_wav = f"{out_dir}/wavs/{index}.wav"
        audio = AudioSegment.from_file(row['file'])
        sample_rate = audio.frame_rate
        samples = np.array(audio.get_array_of_samples())
        cutoff_frequency = np.random.choice([1200,1600,1800,2000,2500,3000,3200,3500])  # Tần số cắt được chọn random trong các mẫu
        filtered_samples = low_pass_filter(samples, cutoff_frequency, sample_rate, order=10)
        filtered_samples = np.clip(filtered_samples, -32768, 32767)
        filtered_samples = filtered_samples.astype(np.int16)
        output_audio = audio._spawn(filtered_samples.tobytes())
        output_audio.export(new_path_wav, format="wav")
        text = row['text']
        duration = row['duration']
        print(f'{new_path_wav},{text},{duration}', file=fp)

0it [00:00, ?it/s]

218it [00:00, 250.41it/s]


In [ ]:

def read_csv_and_extract_columns(csv_file):
    df = pd.read_csv(csv_file)
    if all(col in df.columns for col in ['file', 'text', 'duration']):
        extracted_df = df[['file', 'text', 'duration']]
        return extracted_df
    else:
        raise ValueError("File CSV không chứa đủ các cột 'file', 'text', 'duration'.")

csv_file = '/path/to/your/file.csv'
extracted_data = read_csv_and_extract_columns(csv_file)

print(extracted_data)


# test

In [4]:
import pandas as pd
import numpy as np
from scipy.fft import fft, ifft
from pydub import AudioSegment
import random
import tqdm
import os

# Hàm để cắt ngẫu nhiên các đoạn tần số khác nhau với các khoảng thời gian khác nhau
def zero_out_random_frequencies_in_segments(audio_path, output_path, num_segments, max_duration, freq_ranges):
    # Đọc file âm thanh
    audio = AudioSegment.from_file(audio_path)
    samples = np.array(audio.get_array_of_samples(), dtype=np.float32)
    sample_rate = audio.frame_rate
    n_samples = len(samples)

    # Tạo danh sách các khoảng thời gian ngẫu nhiên
    random_segments = []
    for _ in range(num_segments):
        # Chọn độ dài ngẫu nhiên cho khoảng thời gian
        duration = random.randint(1, max_duration) * sample_rate  # Giây
        if n_samples - duration <= 0:
            # Nếu số mẫu còn lại không đủ, bỏ qua khoảng thời gian này
            continue
        start_time = random.randint(0, n_samples - duration)
        end_time = start_time + duration
        random_segments.append((start_time, end_time))

    # Áp dụng cho từng khoảng thời gian ngẫu nhiên
    for start_time, end_time in random_segments:
        # Tách đoạn tín hiệu cần xử lý
        segment_to_process = samples[start_time:end_time]

        # Thực hiện biến đổi Fourier
        fft_data = fft(segment_to_process)

        # Chọn ngẫu nhiên dải tần số để làm biên độ bằng 0
        low_freq, high_freq = random.choice(freq_ranges)

        # Tính toán các chỉ số tương ứng với dải tần số
        n = len(segment_to_process)
        freqs = np.fft.fftfreq(n, d=1/sample_rate)
        low_idx = np.abs(freqs - low_freq).argmin()
        high_idx = np.abs(freqs - high_freq).argmin()

        # Làm bằng 0 biên độ trong dải tần số mong muốn
        fft_data[low_idx:high_idx] = 0
        fft_data[-high_idx:-low_idx] = 0  # Đối xứng

        # Chuyển đổi ngược sang miền thời gian
        filtered_segment = np.real(ifft(fft_data))

        # Đảm bảo giá trị của tín hiệu nằm trong khoảng int16
        filtered_segment = np.clip(filtered_segment, -32768, 32767)

        # Thay thế đoạn tín hiệu ban đầu bằng đoạn đã lọc
        samples[start_time:end_time] = filtered_segment

    # Chuyển đổi tín hiệu trở lại dạng int16
    samples = samples.astype(np.int16)

    # Ghi file âm thanh đã xử lý
    output_audio = audio._spawn(samples.tobytes())
    output_audio.export(output_path, format="wav")

# Đường dẫn tới file CSV và thư mục xuất kết quả
csv_file = "/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/fubon_11_10_2024/metadata.csv"
out_dir = "/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/augmant_lost_signal_full"

# Đọc file CSV
df_fubong = pd.read_csv(csv_file)

# Tạo thư mục đầu ra nếu chưa tồn tại
os.makedirs(f'{out_dir}/wavs', exist_ok=True)

# Tạo file metadata mới và xử lý từng dòng trong CSV
with open(f'{out_dir}/metadata.csv', 'w', encoding='utf8') as fp:
    print(f'file,text,duration', file=fp)
    
    # Lặp qua từng hàng trong file CSV
    for index, row in tqdm.tqdm(df_fubong.iterrows(), total=df_fubong.shape[0]):
        new_path_wav = f"{out_dir}/wavs/{index}.wav"
        num_segments = random.randint(3, 7)  # Số lượng khoảng thời gian ngẫu nhiên
        max_duration = 2  # Độ dài tối đa mỗi khoảng thời gian (giây)
        text = row['text']
        duration = row['duration']
        freq_ranges = [(2000, 3000), (800, 1600), (100,200), (400,600), (100,400), (200,800), (700,1500), (600,1000), (1000,1700), (1700,2300), (2300,2500), (2500,3000), (3000,3500)]  # Các dải tần số

        # Gọi hàm để xử lý file âm thanh và lưu kết quả
        zero_out_random_frequencies_in_segments(row['file'], new_path_wav, num_segments, max_duration, freq_ranges)

        # Ghi vào file metadata mới
        print(f'{new_path_wav},{text},{duration}', file=fp)


100%|██████████| 218/218 [00:00<00:00, 782.01it/s]


In [6]:
import os
import pandas as pd

csv_directory = "/home/pdnguyen/Audio_Augmentation/fubon_raw"

all_data = pd.DataFrame()

for root, dirs, files in os.walk(csv_directory):
    for file in files:
        if file.endswith(".csv"):
            csv_file_path = os.path.join(root, file)
            
            try:
                data = pd.read_csv(csv_file_path)
                all_data = pd.concat([all_data, data], ignore_index=True)
                print(f"Đã đọc file: {csv_file_path}")
            except Exception as e:
                print(f"Không thể đọc file: {csv_file_path} do lỗi: {e}")

all_data = all_data.sample(frac=1).reset_index(drop=True)

output_csv_path = "/home/pdnguyen/Audio_Augmentation/fubon_raw/metadata.csv"
all_data.to_csv(output_csv_path, index=False, encoding='utf-8')
print(f"Tất cả các file CSV đã được nối, trộn đều và lưu vào: {output_csv_path}")


Đã đọc file: /home/pdnguyen/Audio_Augmentation/fubon_raw/fubon_11_10_2024/metadata.csv
Đã đọc file: /home/pdnguyen/Audio_Augmentation/fubon_raw/augment_lost_signal/metadata.csv
Đã đọc file: /home/pdnguyen/Audio_Augmentation/fubon_raw/augmant_lost_signal_full/metadata.csv
Đã đọc file: /home/pdnguyen/Audio_Augmentation/fubon_raw/fubon_bandpass_filter/metadata.csv
Tất cả các file CSV đã được nối, trộn đều và lưu vào: /home/pdnguyen/Audio_Augmentation/fubon_raw/metadata.csv


In [1]:
import pandas as pd

def concatenate_and_shuffle_csv(csv_paths, output_csv_path):
    # Đọc và ghép tất cả các file CSV lại với nhau
    df_list = [pd.read_csv(csv_path) for csv_path in csv_paths]
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Trộn đều DataFrame
    shuffled_df = combined_df.sample(frac=1).reset_index(drop=True)
    
    # Lưu kết quả vào file CSV mới
    shuffled_df.to_csv(output_csv_path, index=False)

# Mảng chứa các đường dẫn tới file CSV
csv_paths = [
    '/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/fubon_raw/metadata.csv',
    '/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/fubon_raw_v1/metadata.csv',
    '/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/fubon_raw_v2/metadata.csv',
    '/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/fubon_raw_v3/metadata.csv',
    '/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/fubon_raw_v4/metadata.csv',
    '/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/fubon_raw_v5/metadata.csv'
]

# Đường dẫn file CSV xuất ra sau khi ghép và trộn đều
output_csv_path = '/home/pdnguyen/Audio_Augmentation/73.fubon_self_eval_14_10/73.metadatafubon_14_10.csv'

# Gọi hàm để ghép và trộn đều file
concatenate_and_shuffle_csv(csv_paths, output_csv_path)


In [14]:
import pandas as pd
from sklearn.utils import shuffle

def shuffle_and_save_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    
    df_shuffled = shuffle(df)
    
    df_shuffled.to_csv(output_csv_path, index=False)
input_csv_path = '/mnt/driver/STT_data/STT_dataset/72.fubon_10_10_2024/metadata.csv'
output_csv_path = '/mnt/driver/STT_data/STT_dataset/72.fubon_10_10_2024/metadata_real.csv'
shuffle_and_save_csv(input_csv_path, output_csv_path)
